In [1]:
import cv2
import multiprocessing as mp
import time

In [ ]:
def check_available_cameras(max_cam=8):
    '''
        Check available idx cameras
    '''
    available = []
    for i in range(max_cam):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            ret, frame = cap.read()
            if ret:
                available.append(i)
                print(f'Camera {i} is available, shape: {frame.shape[1]}x{frame.shape[0]}')
            else:
                print(f'Camera is opened but not get frame!!!')
            cap.release()
        else:
            print(f'Camera {i} not available(...')
    
    return available


def camera_capture(idx_cam: int, queue: mp.Queue, stop_event: mp.Event): # type: ignore
    """
        Process of capturing video from a singles camera,
        processed while not will be set flag 'stop_event'
    """

    cap = cv2.VideoCapture(idx_cam)
    if not cap.isOpened():
        print(f'Error not open camera {idx_cam}')
        return
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    if fps == 0: fps = 30

    print(f'[Cam_{idx_cam}] Play: {width}x{height}@{fps} FPS')

    while True:
        ret, frame = cap.read()
        if not ret:
            print(f'Camera {idx_cam} error read frame!')
            break
        # added text in frame
        text = f'Cam {idx_cam} | {width}x{height}'
        cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0))

        # Sent frame if queue not full
        if queue.empty():
            queue.put(frame)
        else:
            queue.get() # clearing queue if not it empty
            queue.put(frame)

        time.sleep(1 / (fps * 2)) # delay for economy cpu
    cap.release()
    print(f'Cam {idx_cam} process end!')

